
Load python packages.

In [2]:
# I think a lot of these are unnecessary!
import pandas as pd
import numpy as np
import json
import requests
import shutil
import os
import gzip
import copernicusmarine as cpm
import re
import ast
from pathlib import Path
import re
from tqdm import tqdm
from urllib.request import urlretrieve
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from datetime import datetime
from scipy import interpolate
from pprint import pprint


c:\Users\Work\OneDrive - NOC\Documents\PARTITRICS\PARTITRICS_Python_Hans\PARTITRICS_conda_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Create directories to store data

In [3]:
def create_missing_directories():
    # Define the path to the parent directory
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

    # Check if 'data' folder exists in the parent directory
    data_dir = os.path.join(parent_dir, 'data')
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        print("'data' folder created in the parent directory.")
    else:
        print("'data' folder already exists in the parent directory.")

    # Check if 'satellite' directory exists inside 'data' folder
    satellite_dir = os.path.join(data_dir, 'satellite')
    if not os.path.exists(satellite_dir):
        os.makedirs(satellite_dir)
        print("'satellite' directory created inside 'data' folder.")
    else:
        print("'satellite' directory already exists inside 'data' folder.")

if __name__ == "__main__":
    create_missing_directories()

'data' folder already exists in the parent directory.
'satellite' directory already exists inside 'data' folder.


If it is the first time running copernicusmarine module on this machine, login using the terminal with:
<code><pre>copernicusmarine login</code></pre>
And enter your credentials. You will need an account already to do this. (https://data.marine.copernicus.eu/register) </br>
This will store a credential file locally, you should only have to do this once per machine.

When changing the catalogue search term, use the name of the product you'd like, e.g., from this domain, (https://data.marine.copernicus.eu/product/OCEANCOLOUR_ATL_BGC_L3_NRT_009_111/services), any of the named Dataset ID's

In [6]:
# 
catalogue_search_term = "cmems_obs-oc_atl_bgc-plankton_nrt_l3-olci-300m_P1D"

satellite_catalogue = cpm.describe(contains=[catalogue_search_term], include_datasets=True, overwrite_metadata_cache=False) # Last command set to true ensures cache is up to date, i.e. browsing the latest catalogue. 
dataset_to_sample = satellite_catalogue['products'][0]['datasets'][0] # Define dataset to sample for next code block.
vars_in_dataset   = [f"{[variable['short_name'] for variable in service['variables']]}" for service in dataset_to_sample['versions'][0]['parts'][0]['services']]

dataset_to_extract = list(dataset_to_sample.values())
dataset_to_extract = dataset_to_extract[0]

vars_in_dataset = vars_in_dataset[0]
vars_in_dataset_as_list = ast.literal_eval(vars_in_dataset)
print(vars_in_dataset_as_list)
var_to_extract = vars_in_dataset_as_list[0] # Index the variables you want

['CHL', 'CHL_uncertainty', 'flags']


Note directories you wish to store data in, set bounds for time, space, depth

In [7]:
parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')

variables_to_extract = [var_to_extract] # Input for variable must be list

start_time = "2024-04-01T00:00:00" # Format time as YYYY-MM-DD"T"HH:MM:SS ,without T in parenthesis
end_time   = "2024-05-04T23:59:59"

min_depth = 0 # Primarily interested in surface values only. Will flag if outside of range of product
max_depth = 1

min_lon = -35 # Same lat/lons from autonomy visualisation, can be freely changed
max_lon = -5
min_lat = 55
max_lat = 66

saved_netCDF_as = "CMEMS_Iceland_Basin_CHLA_April-May2024.nc" # This changes the name of the output file.

In [8]:
cpm.subset(dataset_id=dataset_to_extract,
  variables=variables_to_extract,
  minimum_longitude= min_lon,
  maximum_longitude= max_lon,
  minimum_latitude= min_lat,
  maximum_latitude= max_lat,
  start_datetime=start_time,
  end_datetime=end_time,
  minimum_depth=min_depth,
  maximum_depth=max_depth,
  output_filename = saved_netCDF_as,
  output_directory = satellite_dir)

INFO - 2024-05-10T12:53:28Z - Dataset version was not specified, the latest one was selected: "202303"
INFO - 2024-05-10T12:53:28Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-05-10T12:53:36Z - Service was not specified, the default one was selected: "arco-geo-series"
WARNING - 2024-05-10T12:53:42Z - Some or all of your subset selection [55, 66] for the latitude dimension  exceed the dataset coordinates [20.002777099609375, 65.99722290039062]
INFO - 2024-05-10T12:53:42Z - Downloading using service arco-geo-series...
INFO - 2024-05-10T12:53:49Z - <xarray.Dataset> Size: 1GB
Dimensions:    (time: 34, latitude: 1980, longitude: 5400)
Coordinates:
  * latitude   (latitude) float32 8kB 55.0 55.01 55.01 ... 65.99 65.99 66.0
  * longitude  (longitude) float32 22kB -35.0 -34.99 -34.99 ... -5.008 -5.003
  * time       (time) datetime64[ns] 272B 2024-04-01 2024-04-02 ... 2024-05-04
Data variables:
    CHL        (time, latitude, longitude) float32 1GB dask.a

100%|██████████| 7550/7550 [00:14<00:00, 537.04it/s]

INFO - 2024-05-10T12:55:16Z - Successfully downloaded to c:\Users\Work\OneDrive - NOC\Documents\PARTITRICS\PARTITRICS_Python_Hans\biocarbon_nrt_data_viz\data\satellite\CMEMS_Iceland_Basin_CHLA_April-May2024.nc


WindowsPath('c:/Users/Work/OneDrive - NOC/Documents/PARTITRICS/PARTITRICS_Python_Hans/biocarbon_nrt_data_viz/data/satellite/CMEMS_Iceland_Basin_CHLA_April-May2024.nc')